# Sun Position and Timestamp sanity check

Comparing Richmond results with tracking for Single-hour tracking model using Gendaylit2 in Radiance, to View factor.
CW: 1 m
HubHeight: 1m
Rtr: 1.5m
Albedo: 0.62
Limit angle: 60 deg.

## Time Delta
In this bifacial_radiance version, gendaylit2 calculates sun position with a time delta of -30minutes for the TMY3 data.
Tracker position gets also claculated with that same time delta of -30 minutes.


# Radiance part

In [1]:
import sys

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance import *
    print " Successful import of bifacial_radiance version . "
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.


 Successful import of bifacial_radiance version . 


In [7]:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\SunPosTest'
TMYtoread = "C:\Users\sayala\Documents\GitHub\\bifacialvf\\bifacialvf\data\\724010TYA.csv"   # VA Richmond

# MakeModule Parameters
module_type='squarepanel'
module_height = 1  # 2-up portrait Longi with 15cm additional gap
module_width = 1
orientation='portrait' 
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 1   
axis_azimuth = 180.0
roundTrackerAngleBool = False # VIEW FACTOR DOESNT ROUND SO FOR COMARISON TURNIGN THIS OFF. 
angledelta = 5
limit_angle = 60
backtrack = True
    
# SceneDict Parameters
rtr = 1.5
gcr = module_height/rtr   # ground cover ratio,  = module_height / pitch
albedo = 0.62  #'concrete'     # ground albedo
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.

for timestamp in range (7, 17):
    print " ********* Timestamp: ", timestamp
    simulationname = '1axisTrack_'+str(timestamp)
    
    # Running the simulation
    demo2 = RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
    demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
    #epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
    #metdata = demo2.readEPW(epwfile) # read in the weather data
    metdata = demo2.readTMY(TMYtoread)
    demo2.gendaylit2(metdata, timestamp)  # Noon, June 17th

    demo2.makeModule(name=module_type,x=module_width,y=module_height)    
    tracker_theta, tracker_height, azimuth_ang = demo2._getTrackingGeometryTimeIndex(metdata, timeindex=timestamp, angledelta = angledelta, roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, module_height = module_height )
    sceneDict = {'tilt':tracker_theta,'pitch': module_height / gcr,'height':tracker_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  
    scene = demo2.makeScene(module_type, sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
    octfile = demo2.makeOct(demo2.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
    analysis = AnalysisObj(octfile, demo2.basename)  # return an analysis object including the scan dimensions for back irradiance
    frontDict, backDict = analysis.analysis(octfile, demo2.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
    print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )
    print("")

path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Saving file EPWs\tmy3_temp.csv, # points: 8760
Sky generated with Gendaylit 2, with DNI: 2.0, DHI: 9.0
Datetime TimeIndex 1987-01-01 08:00:00-05:00

Module Name: squarepanel
REWRITING pre-existing module file. 
Module squarepanel successfully created
For this timestamp, panels are facing East
Tracker theta has been calculated to 0.113, no rounding performed.
Module clearance height has been calculated to 0.999, for this tracker theta.
Created 1axisTrack_7.oct
Linescan in process: 1axisTrack_7_Front
Linescan in process: 1axisTrack_7_Back
Saved: results\irr_1axisTrack_7.csv
Annual bifacial ratio: 0.215 - 0.216

path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Saving file EPWs\tmy3_temp.csv, # points: 8760
Sky generated with Gendaylit 2, with DNI: 3.0, DHI: 58.0
Datetime TimeIndex 1987-01-01 09:00:00-05:00

Module Name: squarepanel
REWRITING pre-existing module file. 
Module squarepanel successfully created
For this time

For timestamp 17, Sun is already below horizon. 
Datetime TimeIndex 1955-01-01 18:00:00-05:00
Program Crashes. Add a clausule so it skips?

Local Sunrise,Sunset (SPA algorithm): 7.401557,17.035080
https://midcdmz.nrel.gov/apps/spa.pl?syear=2005&smonth=1&sday=1&eyear=2005&emonth=1&eday=1&step=1&stepunit=1&latitude=37.517&longitude=-77.317&timezone=-5&elev=1829&press=835&temp=10&dut1=0.0&deltat=64.797&azmrot=180&slope=0&refract=0.5667&field=4&field=6&zip=0

Which makes sense for 8timestam 18:00, which gets calculated at 17:30, to find sun udnerneath.

# BifacialVF Part


In [5]:
import sys

sys.path.insert(0, '../../bifacialvf/bifacialvf')

import bifacialvf    

#2. Set the Values of your test
# Remember all values are normalized to your panel length (slope, which will equal 1).
# If your slope is different than 1 m, desired C and rtr (or D) will need to be 
# divided by the slope length.
# i.e.: panel 1mx1.59m, in portrait mode means slope = 1.59. For a height C of 1m, C = 1/1.59. 
#         For a rtr of 1.5m, D=0.51519/1.59 if beta tilt angle = 10 

# Set mandatory variables
beta = 10                   # PV tilt (deg)
sazm = 180                  # PV Azimuth(deg)
C = 1                      # GroundClearance(panel slope lengths)
rtr = 1.5              # row to row spacing in panel lengths. 
GCR = 1.0/rtr             # Ground Cover Ratio

writefiletitle = "C:\Users\sayala\Documents\RadianceScenes\SunPosTest\\results\VIEWFACTOR.csv"

# Set optional variables.  These are the default values
rowType = "interior"        # RowType(first interior last single)
transFactor = 0.0         # TransmissionFactor(open area fraction)
cellRows = 6                # CellRows(# hor rows in panel)   This is the number of irradiance values returned along module chord
PVfrontSurface = "glass"    # PVfrontSurface(glass or ARglass)
PVbackSurface = "glass"     # PVbackSurface(glass or ARglass)
albedo = 0.62               # ground albedo beneath system
# 1-axis tracking instructions (optional)
tracking=True
backtrack=True       # backtracking optimization as defined in pvlib
max_angle = 60


bifacialvf.simulate(TMYtoread, writefiletitle, beta, sazm, 
                C=C, rtr=rtr, rowType=rowType, transFactor=transFactor, cellRows=cellRows,
                PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, albedo=albedo, 
                tracking=tracking, backtrack=backtrack, )


 
********* 
Running Simulation for TMY3:  C:\Users\sayala\Documents\GitHub\bifacialvf\bifacialvf\data\724010TYA.csv
Location:   "RICHMOND INTERNATIONAL AP"
Lat:  37.517  Long:  -77.317  Tz  -5.0
Parameters: beta:  0   Sazm:  180   Height:  1   rtr separation:  1.5   Row type:  interior   Albedo:  0.62
Saving into C:\Users\sayala\Documents\RadianceScenes\SunPosTest\results\VIEWFACTOR.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  0.5
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Finished


In [16]:
#4. Load the results from the resultfile
(data, metadata) = bifacialvf.loadVFresults(writefiletitle)
#print data.keys()
# calculate average front and back global tilted irradiance across the module chord
data['GTIFrontavg'] = data[['No_1_RowFrontGTI', 'No_2_RowFrontGTI','No_3_RowFrontGTI','No_4_RowFrontGTI','No_5_RowFrontGTI','No_6_RowFrontGTI']].mean(axis=1)
data['GTIBackavg'] = data[['No_1_RowBackGTI', 'No_2_RowBackGTI','No_3_RowBackGTI','No_4_RowBackGTI','No_5_RowBackGTI','No_6_RowBackGTI']].mean(axis=1)

# Print the annual bifacial ratio
frontIrrSum = data['GTIFrontavg'].sum()
backIrrSum = data['GTIBackavg'].sum()
print('The bifacial ratio for ground clearance {} and rtr spacing {} is: {:.1f}%'.format(C,rtr,backIrrSum/frontIrrSum*100))





C:\Users\sayala\Documents\GitHub\bifacialvf\bifacialvf\data\724010TYA.csv


In [8]:
# PRINT DNI / DHI FOR EPWs

testfolder = r'C:\Users\sayala\Documents\RadianceScenes\SunPosTest'

# MakeModule Parameters
module_type='squarepanel'
module_height = 1  # 2-up portrait Longi with 15cm additional gap
module_width = 1
orientation='portrait' 
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 1   
axis_azimuth = 180.0
roundTrackerAngleBool = False # VIEW FACTOR DOESNT ROUND SO FOR COMARISON TURNIGN THIS OFF. 
angledelta = 5
limit_angle = 60
backtrack = True
    
# SceneDict Parameters
rtr = 1.5
gcr = module_height/rtr   # ground cover ratio,  = module_height / pitch
albedo = 0.62  #'concrete'     # ground albedo
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.

for timestamp in range (7, 17):
    print " ********* Timestamp: ", timestamp
    simulationname = '1axisTrack_'+str(timestamp)
    
    # Running the simulation
    demo2 = RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
    demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
    epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
    metdata = demo2.readEPW(epwfile) # read in the weather data
    demo2.gendaylit2(metdata, timestamp)  # Noon, June 17th


 ********* Timestamp:  7
path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
Sky generated with Gendaylit 2, with DNI: 0.0, DHI: 9.0
Datetime TimeIndex 1955-01-01 08:00:00-05:00
 ********* Timestamp:  8
path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
Sky generated with Gendaylit 2, with DNI: 6.0, DHI: 59.0
Datetime TimeIndex 1955-01-01 09:00:00-05:00
 ********* Timestamp:  9
path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
Sky generated with Gendaylit 2, with DNI: 8.0, DHI: 108.0
Datetime TimeIndex 1955-01-01 10:00:00-05:00
 ********* Timestamp:  10
path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
Sky generated with Gendaylit 2, with DNI: 153.0, DHI: 204.0
Datetime TimeInde

In [2]:
# TIME DIFFERENCe

import time


testfolder = r'C:\Users\sayala\Documents\RadianceScenes\SunPosTest'

# MakeModule Parameters
module_type='squarepanel'
module_height = 1  # 2-up portrait Longi with 15cm additional gap
module_width = 1
orientation='portrait' 
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 1   
axis_azimuth = 180.0
roundTrackerAngleBool = False # VIEW FACTOR DOESNT ROUND SO FOR COMARISON TURNIGN THIS OFF. 
angledelta = 5
limit_angle = 60
backtrack = True
    
# SceneDict Parameters
rtr = 1.5
gcr = module_height/rtr   # ground cover ratio,  = module_height / pitch
albedo = 0.62  #'concrete'     # ground albedo
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.

simulationname = '1axisTrack_'

# Running the simulation
demo2 = RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data

tic = time.time()
for timestamp in range (0, 8759):

    demo2.gendaylit2(metdata, timestamp)  # Noon, June 17th

toc=time.time()
A = toc-tic

tic = time.time()
for timestamp in range (0, 8759):

    demo2.gendaylit(metdata, timestamp)  # Noon, June 17th

toc=time.time()
B = toc-tic

print A, 'sec Elapsed Gendaylit2'
print B, 'sec Elapsed Gendaylit'


path = C:\Users\sayala\Documents\RadianceScenes\SunPosTest
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
127.726000071 sec Elapsed Gendaylit2
3.93400001526 sec Elapsed Gendaylit
